In [1]:
import pandas as pd
import requests
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold, RepeatedKFold, StratifiedKFold
import time 
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import classification_report, confusion_matrix
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
import seaborn as sb

In [2]:
def get_user_animelist(username: str, client_id: str):
    # Loads list of anime from MAL for a given user
    url = f'https://api.myanimelist.net/v2/users/{username}/animelist?limit=500'
    headers = {
        'X-MAL-CLIENT-ID': client_id
    }
    params = {
        'fields': 'id, title, list_status{score,status}, start_season{year}, mean, genres, popularity, media_type, rating, num_episodes, studios, num_list_users,favorites'
    }

    all_data = []
    next_page = url

    while next_page:
        response = requests.get(next_page, headers=headers, params=params if next_page == url else None)
        if response.status_code != 200:
            raise ValueError(f"Error with API request: {response.status_code} - {response.text}")

        data = response.json()
        all_data.extend(data['data'])
        next_page = data.get("paging", {}).get("next")

        print(f"Loaded {len(all_data)} anime...")

    return all_data

In [3]:
def to_dataframe(all_data):
    rows = []
    for item in all_data:
        anime = item['node']
        score = item.get('list_status', {}).get('score')
        status = item.get('list_status', {}).get('status')
        # num_episodes_watched = item.get('list_status', {}).get('num_episodes_watched')
        year = item.get('node', {}).get('start_season', {}).get('year')
        rows.append({
            "id": anime['id'],
            "title": anime['title'],
            "mean": anime.get('mean'),
            "genres": [g['name'] for g in anime.get('genres', [])],
            "studios": [s['name'] for s in anime.get('studios', [])],
            "rating": anime.get('rating'),
            "year": year,
            "type": anime.get('media_type'),
            "popularity": anime.get('popularity'),
            "score": score,
            "status": status,
            "members": anime['num_list_users'],
            "num_episodes": anime['num_episodes'],
        })
    
    df = pd.DataFrame(rows)
    df['studios'] = df['studios'].str.join(", ")
    df['genres'] = df['genres'].str.join(", ")

    return df

In [6]:
def calculate_genre_affinity_simple(df_train):
    genre_cols = [col for col in df_train.columns if col.startswith('Genre_')]
    genre_affinity = {}
    
    overall_mean = df_train['score'].mean()
    
    for genre_col in genre_cols:
        genre_name = genre_col.replace('Genre_', '')
        mask = df_train[genre_col] == 1
        
        if mask.sum() >= 5:
            genre_affinity[genre_name] = df_train[mask]['score'].mean()
        else:
            genre_affinity[genre_name] = overall_mean
    
    return genre_affinity

In [7]:
def calculate_studio_mean(df_train):
    studios_columns = [col for col in df_train.columns if col.startswith('Studios_')]
    studio_mean = {}
    overall_mean = df_train['score'].mean()
    for studio_column in studios_columns:
        studio_name = studio_column.replace('Studios_', '')
        mask = df_train[studio_column] == 1
        
        if mask.sum() >= 3:
            studio_scores = df_train[mask]['score']
            mean_score = studio_scores.mean()
            studio_mean[studio_name] = mean_score
        else:
            studio_mean[studio_name] = overall_mean
            
    return studio_mean

In [8]:
import numpy as np

def map_score_to_class(score):
    if score <= 4:
        return 0  # Bad
    elif score <= 6:
        return 1  # Okay
    elif score <= 8:
        return 2  # Good
    else:
        return 3  # Perfect

In [9]:
import configparser

config = configparser.ConfigParser()
config.read('../config.ini')
USER_NAME = config.get('USER','USER_NAME').strip()
CLIENT_ID = config.get('USER','CLIENT_ID').strip()

In [10]:
data = get_user_animelist(USER_NAME, CLIENT_ID)

Loaded 500 anime...
Loaded 1000 anime...
Loaded 1500 anime...
Loaded 2000 anime...
Loaded 2500 anime...
Loaded 3000 anime...
Loaded 3500 anime...
Loaded 4000 anime...
Loaded 4500 anime...
Loaded 5000 anime...
Loaded 5059 anime...


In [39]:
df = to_dataframe(data)

In [40]:
df['score_class'] = df['score'].apply(map_score_to_class)

In [41]:
df["studios"].replace(to_replace="", value="Unknown", inplace=True)
df["genres"].replace(to_replace="", value="Unknown", inplace=True)
df = df.dropna(subset=['score'])

C:\Users\dobre\AppData\Local\Temp\ipykernel_13212\87429005.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["studios"].replace(to_replace="", value="Unknown", inplace=True)
C:\Users\dobre\AppData\Local\Temp\ipykernel_13212\87429005.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

In [42]:
display(df["score"].value_counts().head(11))

score
0     1333
5      793
7      757
6      574
8      517
4      434
3      350
9      151
2      114
10      25
1       11
Name: count, dtype: int64

In [43]:
df.drop(columns=['title', 'id'], inplace=True)

In [44]:
df = df[df['score'] > 0].copy()
    
df['mean'] = df['mean'].fillna(df.groupby('type')['mean'].transform('median'))
    
categorical_cols = ['type', 'rating', 'status']
for col in categorical_cols:
    df[col] = df[col].fillna('Unknown')

In [45]:
df['year'] = df['year'].fillna(df['year'].mode()[0])
df['num_episodes'] = df['num_episodes'].fillna(df['num_episodes'].mode()[0])

In [46]:
df['popularity'] = df['popularity'].fillna(df['popularity'].median())

In [47]:
df['anime_age'] = 2025 - df['year']
df['anime_age'] = df['anime_age'].astype(int)
df.drop(columns=['year'], inplace=True)

In [48]:
df['log_members'] = np.log1p(df['members'])
df.drop(columns=['members'], inplace=True)

In [49]:
# Get dummy variables for each unique genre
genre_dummies = (
    df['genres']
    .str.split(', ', expand=True) 
    .stack()       
    .str.get_dummies()         
    .groupby(level=0)         
    .sum()                    
    .add_prefix('Genre_')  
)

In [50]:
df = pd.concat([df, genre_dummies], axis=1)

In [51]:
genre_columns = [col for col in df.columns if col.startswith('Genre_')]

genre_counts = df[genre_columns].sum().sort_values(ascending=False)
print(genre_counts)

Genre_Action         1480
Genre_Comedy         1336
Genre_Fantasy         996
Genre_School          918
Genre_Romance         871
                     ... 
Genre_Educational       6
Genre_Hentai            2
Genre_Boys Love         2
Genre_Erotica           2
Genre_Unknown           1
Length: 79, dtype: int64


In [52]:
genre_columns = [col for col in df.columns if col.startswith('Genre_')]

genre_counts = df[genre_columns].sum().sort_values(ascending=False)
total_anime = len(df)

min_count = max(5, int(total_anime * 0.01))
frequent_genres= genre_counts[genre_counts >= min_count].index.tolist()

print(f"studio_counts: {len(genre_counts)}")
print(f"Studios with >= {min_count} anime: {len(frequent_genres)}")

rare_genre_columns = [col for col in genre_columns if col not in frequent_genres]
df['Genre_Other'] = df[rare_genre_columns].max(axis=1) 

df.drop(columns=rare_genre_columns, inplace=True)

studio_counts: 79
Studios with >= 37 anime: 58


In [53]:
df.drop(columns=['genres'], inplace=True)

In [54]:
type_dummies = pd.get_dummies(df['type'], prefix='Type')
df = pd.concat([df, type_dummies], axis=1)
df.drop(columns=['type'], inplace=True)

In [55]:
studio_dummies = (
    df['studios']
    .str.split(', ', expand=True) 
    .stack()       
    .str.get_dummies()         
    .groupby(level=0)         
    .sum()                    
    .add_prefix('Studios_')  
)
df = pd.concat([df, studio_dummies], axis=1)

In [56]:
studio_columns = [col for col in df.columns if col.startswith('Studios_')]

studio_counts = df[studio_columns].sum().sort_values(ascending=False)
print(studio_counts)

Studios_J.C.Staff               191
Studios_A-1 Pictures            147
Studios_Madhouse                136
Studios_Production I.G          122
Studios_Studio Deen             118
                               ... 
Studios_Karaku                    1
Studios_Studio Gram               1
Studios_KeyEast                   1
Studios_Studio Gazelle            1
Studios_Next Media Animation      1
Length: 330, dtype: int64


In [57]:
studio_columns = [col for col in df.columns if col.startswith('Studios_')]

studio_counts = df[studio_columns].sum().sort_values(ascending=False)
total_anime = len(df)

min_count = max(10, int(total_anime * 0.01))
frequent_studios = studio_counts[studio_counts >= min_count].index.tolist()

print(f"studio_counts: {len(studio_counts)}")
print(f"Studios with >= {min_count} anime: {len(frequent_studios)}")

rare_studio_columns = [col for col in studio_columns if col not in frequent_studios]
df['Studio_Other'] = df[rare_studio_columns].max(axis=1) 

df.drop(columns=rare_studio_columns, inplace=True)

studio_counts: 330
Studios with >= 37 anime: 29


In [58]:
df.drop(columns=['studios'], inplace=True)

In [59]:
rating_dummies = pd.get_dummies(df['rating'], prefix='Rating')
df = pd.concat([df, rating_dummies], axis=1)
df.drop(columns=['rating'], inplace=True)

In [60]:
status_dummies = pd.get_dummies(df['status'], prefix='Status')
df = pd.concat([df, status_dummies], axis=1)
df.drop(columns=['status'], inplace=True)

In [62]:
df.drop(columns=['Status_completed'], inplace=True)
df.drop(columns=['Type_tv'], inplace=True)
df.drop(columns=['Rating_pg_13'], inplace=True)

In [63]:
bool_cols = df.select_dtypes('bool').columns

df[bool_cols] = df[bool_cols].astype(int)

In [64]:
# df.drop(columns=['Genre_Unknown'], inplace=True)

In [66]:
corr_with_score = df.corr()['score']
high_corr_cols = corr_with_score[np.abs(corr_with_score) > 0.65].index
high_corr_cols = high_corr_cols.drop(['score','score_class'])
print(high_corr_cols)

Index(['mean'], dtype='object')


In [67]:
df.drop(columns=high_corr_cols, inplace=True)

In [68]:
df['num_episodes'] = np.log1p(df['num_episodes'])

In [69]:
df.drop(columns=['popularity'], inplace=True)
# df.drop(columns=['log_members'], inplace=True)

In [70]:
X = df.drop(columns=['score', 'score_class'])
y = df['score_class']

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [72]:
genre_affinity = calculate_genre_affinity_simple(X_train.join(df['score']))

affinity_features_train = pd.DataFrame({
    f"affinity_{genre}": X_train[f"Genre_{genre}"] * affinity
    for genre, affinity in genre_affinity.items()
}, index=X_train.index)

affinity_features_test = pd.DataFrame({
    f"affinity_{genre}": X_test[f"Genre_{genre}"] * affinity
    for genre, affinity in genre_affinity.items()
}, index=X_test.index)

X_train = pd.concat([X_train, affinity_features_train], axis=1)
X_test = pd.concat([X_test, affinity_features_test], axis=1)

In [73]:
X_train = X_train.drop(columns=[col for col in X_train.columns if col.startswith("Genre_")])
X_test = X_test.drop(columns=[col for col in X_test.columns if col.startswith("Genre_")])

In [311]:
# studios_affinity = calculate_studio_mean(X_train.join(df['score']))
# 
# affinity_features_train = pd.DataFrame({
#     f"affinity_{studio}": X_train[f"Studios_{studio}"] * affinity
#     for studio, affinity in studios_affinity.items()
# }, index=X_train.index)
# 
# affinity_features_test = pd.DataFrame({
#     f"affinity_{studio}": X_test[f"Studios_{studio}"] * affinity
#     for studio, affinity in studios_affinity.items()
# }, index=X_test.index)
# 
# X_train = pd.concat([X_train, affinity_features_train], axis=1)
# X_test = pd.concat([X_test, affinity_features_test], axis=1)

In [312]:
# X_train = X_train.drop(columns=[col for col in X_train.columns if col.startswith("Studios_")])
# X_test = X_test.drop(columns=[col for col in X_test.columns if col.startswith("Studios_")])

In [74]:
X_train.sample(10)

,num_episodes,anime_age,log_members,Type_cm,Type_movie,Type_music,Type_ona,Type_ova,Type_pv,Type_special,...,affinity_Supernatural,affinity_Survival,affinity_Suspense,affinity_Team Sports,affinity_Time Travel,affinity_Urban Fantasy,affinity_Vampire,affinity_Video Game,affinity_Workplace,affinity_Other
2243,2.564949,2,11.560715,0,0,0,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
953,0.693147,9,9.862040,0,0,0,0,0,0,1,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3239,2.564949,15,12.430955,0,0,0,0,0,0,0,...,6.121951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4085,0.693147,4,10.484222,0,0,0,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.275735
359,3.218876,21,11.223241,0,0,0,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4366,2.564949,9,9.807912,0,0,0,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3259,0.693147,7,9.491753,0,0,0,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4481,1.098612,9,11.546853,0,0,0,0,0,0,1,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4576,2.197225,1,9.555206,0,0,0,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2869,1.609438,21,8.288786,0,0,0,0,1,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [313]:
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [75]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(enumerate(class_weights))

In [76]:
print(class_weight_dict)

{0: 1.0247592847317744, 1: 0.6816102470265325, 2: 0.7311089303238469, 3: 5.283687943262412}


In [77]:
model = CatBoostClassifier(early_stopping_rounds=50,random_state=42, verbose=0)

In [78]:
params = {
    'iterations': [200, 300, 400],
    'depth': [3, 4, 6],
    'learning_rate': [0.01, 0.03, 0.05],
    'l2_leaf_reg': [3, 5, 7],
    'random_strength': [1.0, 1.5]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)
grid = GridSearchCV(
    estimator=model,
    param_grid=params,
    scoring='f1_macro',
    cv=cv,
    refit=True,
    n_jobs=-1,
    verbose=2,
    return_train_score=True  
)